In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
matplotlib.use("MacOSX")  # 🔹 
import ipywidgets as widgets
from scipy.integrate import solve_ivp
from IPython.display import display, Video, clear_output
import warnings

warnings.filterwarnings("ignore", category=UserWarning, 
                        message="Animation was deleted without rendering anything.")

# ===== PARÁMETROS DEL SISTEMA =====
num_cuentas = 8
longitud_soga = 10.0
masa_cuenta = 1.0
masa_extremos = 1000.0
k_resorte = 10.0
dt = 0.05

# ===== ECUACIONES DIFERENCIALES =====
def ecuaciones_movimiento(t, y):
    pos = y[:num_cuentas*2].reshape(num_cuentas, 2)
    vel = y[num_cuentas*2:].reshape(num_cuentas, 2)
    
    fuerzas = np.zeros((num_cuentas, 2))
    for i in range(num_cuentas):
        if i > 0:
            fuerzas[i, 1] -= k_resorte * (pos[i, 1] - pos[i-1, 1])
        if i < num_cuentas-1:
            fuerzas[i, 1] -= k_resorte * (pos[i, 1] - pos[i+1, 1])
    
    masa = np.full(num_cuentas, masa_cuenta)
    masa[[0, -1]] = masa_extremos
    return np.concatenate([vel.ravel(), (fuerzas / masa[:, None]).ravel()])

# ===== FUNCIÓN PARA CREAR SERruCHOS =====
def crear_serrucho(x0, y0, x1, y1, num_dientes=15):
    x = np.linspace(x0, x1, num_dientes*2)
    y = np.linspace(y0, y1, num_dientes*2)
    for i in range(1, len(x)-1, 2):
        y[i] += 0.15 if i%4 == 1 else -0.15
    return x, y

# ===== CONFIGURACIÓN INTERACTIVA =====
fig, ax = plt.subplots(figsize=(10, 4))
ax.set_xlim(0, longitud_soga)
ax.set_ylim(-2, 2)
ax.set_title("Simulación de Soga con Resortes Serrados")
plt.close(fig)

# Elementos gráficos
lineas = [ax.plot([], [], 'k-', lw=1.5, zorder=1)[0] for _ in range(num_cuentas-1)]
puntos = ax.scatter([], [], s=150, c='blue', zorder=10)

# ===== WIDGETS =====
controles_pos = [widgets.FloatSlider(
    min=-1.5, max=1.5, step=0.1,
    description=f'Pos {i+1}:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
) for i in range(num_cuentas)]

controles_vel = [widgets.FloatSlider(
    min=-1.5, max=1.5, step=0.1,
    description=f'Vel {i+1}:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
) for i in range(num_cuentas)]

boton_animar = widgets.Button(
    description="▶️ Iniciar Animación", 
    button_style='success',
    layout=widgets.Layout(width='250px', height='45px')
)

boton_video = widgets.Button(
    description="🎥 Generar Video", 
    button_style='info',
    layout=widgets.Layout(width='250px', height='45px')
)

salida = widgets.Output()

# ===== FUNCIONES DE ACTUALIZACIÓN =====
def actualizar_animacion(frame):
    pos = sol.y[:num_cuentas*2, frame].reshape(num_cuentas, 2)
    
    for j, linea in enumerate(lineas):
        x_vals, y_vals = crear_serrucho(pos[j,0], pos[j,1], pos[j+1,0], pos[j+1,1])
        linea.set_data(x_vals, y_vals)
    
    puntos.set_offsets(pos)
    return lineas + [puntos]

def generar_video(b):
    with salida:
        clear_output(wait=True)
        print("⏳ Generando video... (30 segundos aprox)")
        
        # Configuración de video
        fig_video, ax_video = plt.subplots(figsize=(10, 4))
        ax_video.set_xlim(0, longitud_soga)
        ax_video.set_ylim(-2, 2)
        
        lineas_video = [ax_video.plot([], [], 'k-', lw=1.5, zorder=1)[0] for _ in range(num_cuentas-1)]
        puntos_video = ax_video.scatter([], [], s=150, c='blue', zorder=10)
        
        # Resolver ecuaciones
        pos = np.zeros((num_cuentas, 2))
        pos[:,0] = np.linspace(0, longitud_soga, num_cuentas)
        vel = np.zeros((num_cuentas, 2))
        
        for i in range(num_cuentas):
            pos[i,1] = controles_pos[i].value
            vel[i,1] = controles_vel[i].value
        
        y0 = np.concatenate([pos.ravel(), vel.ravel()])
        sol_video = solve_ivp(ecuaciones_movimiento, [0, 10], y0, t_eval=np.linspace(0, 10, 300))
        
        def actualizar_video(frame):
            pos_frame = sol_video.y[:num_cuentas*2, frame].reshape(num_cuentas, 2)
            
            for j, linea in enumerate(lineas_video):
                x_vals, y_vals = crear_serrucho(pos_frame[j,0], pos_frame[j,1], pos_frame[j+1,0], pos_frame[j+1,1])
                linea.set_data(x_vals, y_vals)
            
            puntos_video.set_offsets(pos_frame)
            return lineas_video + [puntos_video]
        
        # Generar y guardar video
        ani_video = animation.FuncAnimation(
            fig_video, actualizar_video,
            frames=len(sol_video.t),
            blit=True
        )
        
        ani_video.save('soga_serruchos.mp4', 
                      writer=animation.FFMpegWriter(fps=30, bitrate=5000),
                      dpi=100)
        #plt.close(fig_video)
        
        #display(Video("soga_serruchos.mp4", embed=True))
        print("✅ Video generado correctamente!")

def iniciar_animacion(b):
    global sol, ani
    with salida:
        clear_output(wait=True)

        if 'ani' in globals():
            ani.event_source.stop()
            del ani

        pos = np.zeros((num_cuentas, 2))
        pos[:,0] = np.linspace(0, longitud_soga, num_cuentas)
        vel = np.zeros((num_cuentas, 2))

        for i in range(num_cuentas):
            pos[i,1] = controles_pos[i].value
            vel[i,1] = controles_vel[i].value

        y0 = np.concatenate([pos.ravel(), vel.ravel()])
        sol = solve_ivp(ecuaciones_movimiento, [0, 15], y0, t_eval=np.arange(0, 15, dt))

        ani = animation.FuncAnimation(
            fig, actualizar_animacion,
            frames=len(sol.t),
            interval=dt*1000,
            blit=False  # ⚠ IMPORTANTE: `blit=True` no funciona en todos los entornos
        )
        plt.show(block=True)  # 🔹 Esto obliga a abrir una ventana externa
        plt.pause(0.1)

        print("Animación activa - Ajuste los controles y vuelva a pulsar")


# ===== CONFIGURACIÓN FINAL =====
boton_animar.on_click(iniciar_animacion)
boton_video.on_click(generar_video)

grid = widgets.GridBox(
    children=controles_pos + controles_vel + [boton_animar, boton_video],
    layout=widgets.Layout(
        grid_template_columns="repeat(4, 300px)",
        grid_gap="15px",
        padding="20px"
    )
)

display(widgets.VBox([
    widgets.HTML(
        """<h1 style='color: #2e86c1; text-align: center; margin-bottom: 20px'>
        Simulador de Soga con Resortes Serrados</h1>"""
    ),
    grid,
    salida
]))

: 